# HarvestStat Data Profiling - Sierra Leone

In [1]:
import glob, json
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from tools import save_npz, PrintAdminUnits, PlotAdminShapes
from tools import FDW_PD_Sweeper, FDW_PD_AvalTable, FDW_PD_Compiling, FDW_PD_ValidateFnidName
from tools import FDW_PD_CreateAdminLink, FDW_PD_RatioAdminLink, FDW_PD_ConnectAdminLink
from tools import FDW_PD_CaliSeasonYear, FDW_PD_MergeCropProductionSystem
from tools_graphic import PlotBarProduction, PlotLinePAY
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

In [2]:
# Load FEWS NET administrative boundaries ------- #
epsg = 'epsg:32629' # Sierra Leone
fn_shapes = sorted(glob.glob('../data/shapefile/fewsnet/SL_Admin?_????.shp'))
shape_all = []
for fn in fn_shapes:
    name = fn[-18:-4]
    exec('%s = gpd.read_file("%s").to_crs("%s")' % (name, fn, epsg))
    exec('%s["area"] = %s["geometry"].area/10**6' % (name, name))
    exec('shape_all.append(%s)' % (name))
shape_all = pd.concat(shape_all, axis=0).reset_index(drop=True)
PrintAdminUnits(shape_all)
# ----------------------------------------------- #

# FDW API host address -------------------------- #
host = 'https://fdw.fews.net'
#auth = tuple(json.loads(open('token.json', "r").read()))

parameters = {
    'format': 'json',
    'country':  'Sierra Leone',
    'product': ['R011','R012','R013','R014','R015','R017','R018'],
    'survey_type': 'crop:best'
}
endpoint = '/api/cropproductionindicatorvalue/'
response = requests.get(host + endpoint, params=parameters, proxies={})#auth=auth, 
response.raise_for_status()
df = pd.DataFrame.from_records(response.json())
df_origin = df.copy()
# ----------------------------------------------- #

- FEWS NET admin shapefiles ------------------- #
| year	 | Admin1   | # units   | Admin2   | # units   | Admin3   | # units   |
| 1960	 | SL1960A1 | 4	| SL1960A2	| 14	| nan	| 0	|
| 2010	 | SL2010A1 | 4	| SL2010A2	| 14	| nan	| 0	|
| 2017	 | nan | 0	| SL2017A2	| 16	| nan	| 0	|
----------------------------------------------- #


In [3]:
# Manual Pre-processing before Sweeping --------- #
# 1. Default setting 
# a) None-type crop production system
df.loc[df['crop_production_system'].isna(), 'crop_production_system'] = 'none'
df.loc[df['crop_production_system'] == '', 'crop_production_system'] = 'none'
# b) None-type population group
df.loc[df['population_group'].isna(), 'population_group'] = 'none'
df.loc[df['population_group'] == '', 'population_group'] = 'none'
# Replace season name
df['season_name'] = df['season_name'].replace({
    'Main season': "Main"
})
# ----------------------------------------------- #
df = df[np.isfinite(df.value)]
df_admin1=df[df['admin_2'].isna()]
df = df.dropna(subset=['admin_2'])

# FDW Production Data Inspection ---------------- #
df, df_raw = FDW_PD_Sweeper(df, area_priority="Area Planted")
table_dict = FDW_PD_AvalTable(df, shape_all)
# ----------------------------------------------- #

# FEWS NET Shapefile comparison ----------------- #
shape_used = pd.concat([SL_Admin2_1960], axis=0)
PlotAdminShapes(shape_used, label=True)
# ----------------------------------------------- #

- Remove missing records ---------------------- #
Orignial data points: 475
244/244 "Quantity Produced" points are retained.
195/195 "Area Harvested" points are retained.
36/36 "Yield" points are retained.
Current data points: 475

- Minor changes are applied ------------------- #

- Basic information --------------------------- #
Data period: 1986 - 2016
12 grain types are found: Banana (unspecified), Cashew (unshelled), Cassava, Groundnuts (In Shell), Maize (Corn), Millet, Okras (Fresh), Potato (unspecified), Rice (Paddy), Sesame Seed, Sorghum, Sweet Potatoes
1 seasons are found: Main (05-01)
1 crop production system are found: none
Data sources include:
[1] Ministry of Agriculture, Forestry and Food Security, Sierra Leone --- Crop production survey, Sierra Leone
[2] Ministry of Agriculture, Forestry and Food Security, Sierra Leone --- FEWS NET Agro Maps, Sierra Leone
Administrative-1 fnids: 0
Administrative-2 fnids: 13
0 reporting units are found: 

- Total production over time ----

## Sierra Leone
Sierra Leone crop seasonal calendar

![FEWS NET](https://fews.net/sites/default/files/styles/large_width_880/public/2023-03/seasonal-calendar-sierre-leone.png?itok=Cba4Gu41)

![USDA](https://ipad.fas.usda.gov/rssiws/al/crop_calendar/images/wafrica_sl_calendar.png)

- comparison of boundaries</br>
<img src="../figures/SL_admin_shapes.png"  width="50%" height="50%">

- In Sierra Leone, basd on the FEWS NET data, there were two times of changes in administrative units. We will use the 1960 administrative units



| Year  | Admin-1  | # units  | Admin-2  | # units | Admin-3  | # units |
| :---: | :----:   | :----:   | :----:   | :---:   | :----:   | :---:   |
| 1960	 | SL1960A1 | 4	| SL1960A2	| 14	| nan	| 0	|
| 2010	 | SL2010A1 | 4	| SL2010A2	| 14	| SL2010A3	| 166	|
| 2017	 | nan | 0	| SL2017A2	| 16	| nan	| 0	

In [4]:
# Define the latest shapefile ------------------- #
latest_level = 2
shape_latest = SL_Admin2_1960.copy().to_crs('epsg:32629')
# ----------------------------------------------- #

# Validation of FNIDs and Names ----------------- #
df = FDW_PD_ValidateFnidName(df, shape_used, shape_latest)
# ----------------------------------------------- #

# FDW Production Data Compiling ----------------- #
area, prod = FDW_PD_Compiling(df, shape_used)
area_all, prod_all = area.copy(), prod.copy()
mdx_pss = area.columns.droplevel([0,1]).unique()
# ----------------------------------------------- #
# Link admin boundaries ------------------------- #
link_1960, over_1960 = FDW_PD_CreateAdminLink(SL_Admin2_1960, SL_Admin2_1960, 'ADMIN2', 'ADMIN2', prod, epsg)

link_ratio = FDW_PD_RatioAdminLink(link_1960, prod, over_1960, mdx_pss)

# Connect data with AdminLink
area_new, prod_new = FDW_PD_ConnectAdminLink(link_ratio, area, prod, validation=False)
# ----------------------------------------------- #


CBR is considered for 'SL1960A20402' as no record found in: ['SL1960A20402']


In [5]:
# Manual correction ----------------------------- #
crop_new = prod_new/area_new

# Complete long format DataFrame ---------------- #
df_area = area_new.T.stack().rename('value').reset_index()
df_area['indicator'] = 'area'
df_prod = prod_new.T.stack().rename('value').reset_index()
df_prod['indicator'] = 'production'
df_yield = crop_new.T.stack().rename('value').reset_index()
df_yield['indicator'] = 'yield'
stack = pd.concat([df_area, df_prod, df_yield], axis=0)
# Add "planting year"
cols = ['season_name','product','crop_production_system','planting_month','harvest_year','harvest_month','planting_year']
season_table = df[cols].drop_duplicates()
stack = stack.merge(season_table, on=cols[:-1])
# Add country and admin names
stack = stack.merge(df[['fnid','country','country_code','admin_1','admin_2']].drop_duplicates(), on='fnid', how='inner')
names = [
    'fnid','country','country_code','admin_1','admin_2','name',
    'product','season_name','planting_year','planting_month','harvest_year','harvest_month',
    'crop_production_system','indicator','value'
]
stack = stack[names]
# ----------------------------------------------- #

# Final Processing ------------------------------ #
# Manual change of product name
product_category_custom = {   
#    'Banana (unspecified)': 'Banana',   
#    'Cashew (unshelled)': 'Cashew',
    'Cassava': 'Cassava',
    'Groundnuts (In Shell)': 'Groundnuts (In Shell)',
    'Maize (Corn)': 'Maize',
#    'Millet': 'Millet',
    'Okras (Fresh)': 'Okras',
#    'Potato (unspecified)': 'Potato',
    'Rice (Paddy)': 'Rice',
    'Sorghum': 'Sorghum',
    'Sweet Potatoes': 'Sweet Potatoes',
    'Sesame Seed': 'Sesame Seed'
}

assert set(stack['product'].unique()) == set(product_category_custom.keys())
stack['product'] = stack['product'].replace(product_category_custom)
for f, r in link_ratio.items():
    mdx = pd.MultiIndex.from_frame(r.index.to_frame().reset_index(drop=True).replace(product_category_custom))
    r.index = mdx
    link_ratio[f] = r
# Calibration of crop calendar using External Crop Calendar (ECC)
ecc = pd.read_csv('../data/crop_calendar/external_crop_calendar.csv')
ecc = ecc[ecc['country'] == 'Sierra Leone']
stack, link_ratio = FDW_PD_CaliSeasonYear(stack, ecc, link_ratio)
# None admin names
stack.loc[stack['admin_1'].isna(), 'admin_1'] = 'none'
stack.loc[stack['admin_2'].isna(), 'admin_2'] = 'none'
# ----------------------------------------------- #

#define a QC flag field
# 0 = no flag
# 1 = outlier
stack['QC_flag'] = 0

# Save data
fn_out = '../data/crop/adm_crop_production_SL.csv'
stack.to_csv(fn_out); print(f'{fn_out} is saved.')
save_npz('../data/crop/adm_crop_production_SL_ratio.npz', link_ratio)

All [season_name, product, crop_production_system] are in the external crop calendar.
../data/crop/adm_crop_production_SL.csv is saved.
../data/crop/adm_crop_production_SL_ratio.npz is saved.


# Visualization of production data

In [6]:
# Bar chart of national crop production
country_iso, country_name = 'SL', 'Sierra Leone'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_order = ['Maize', 'Millet', 'Rice', 'Sorghum']
for season_name   in ['Main']:
    footnote = 'National crop production in %s - %s' % (country_name, season_name)
    fn_save = '../figures/%s_bar_natgrainprod_%s.png' % (country_iso, season_name)
    sub = df[df['season_name'] == season_name]
    fig = PlotBarProduction(sub, year, product_order, footnote)#, fn_save)
    fig.show()

In [7]:
# Lineplot of Production-Area-Yield time-series
country_iso, country_name = 'SL', 'Sierra Leone'
df = pd.read_csv('../data/crop/adm_crop_production_%s.csv' % country_iso, index_col=0)
df['year'] = df['harvest_year']
year = [df['year'].min(), df['year'].max()]
product_season = [
    ['Maize','Main'],
    ['Sorghum','Main'],
    ['Rice','Main'],
    ['Groundnuts (In Shell)','Main'],
    ['Cassava','Main'],
]
for product_name, season_name in product_season:
    footnote = 'Production-Area-Yield time-series of %s - %s - %s' % (country_iso, product_name, season_name)
    fn_save = '../figures/%s_line_pay_%s_%s.png' % (country_iso, product_name, season_name)
    sub = df[(df['product'] == product_name) & (df['season_name'] == season_name)]
    fig = PlotLinePAY(sub, year, footnote)#, fn_save)
    fig.show()